In [1]:
%run functions.ipynb

Ready: 2023-05-11 01:23:06


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark')\
        .getOrCreate()
spark

In [ ]:
def repack_file(file_name):
    new_name = file_name.replace('\\', '\\\\').replace('.csv', '_repacked.csv')
    print(new_name)
    start_t = time.time()
    df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', StringType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
    df1 = spark.read.csv(file_name, header='true', schema=df_schema).selectExpr('rubricName','substr(time, 12, 8) as time','date','tx', 'hash(user) user').where('rubricName is not null')
    df1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode('overwrite').csv(new_name)  
    end_t = time.time()
    print(end_t-start_t)

df = dd.read_csv(files_list[0][0])

df.head()

df.to_csv('df.csv', index=False)

repack_file('df.csv')

Пример форматирования строки:
'За %g года я купил %d %s.' % (3.5, 7, 'верблюдов’)
'{:020}'.format(1)

Пример получения текущего рабочего каталога:
import os
cwd = os.getcwd() # текущий каталог
os.listdir() #список файлов и каталогов текущего каталога
os.walk(dirname) # все вложенные файлы и папки
os.path.join(dir_name, file) # объединяем имя каталога и файла

Пример проверки файлов/директорий:
os.path.exists() #существует 
os.path.isdir() #каталог или нет 
os.path.isfile() #файл или нет

Пример копирования/перемещения файлов:
import shutil
original = r'original path where the file is currently stored\file name.file extension'
target = r'target path where the file will be copied\file name.file extension'
shutil.copy(original, target)
shutil.move(original, target)


import shutil

shutil.move()

os.remove() removes a file.

os.rmdir() removes an empty directory.

shutil.rmtree() deletes a directory and all its contents.

In [1]:
df_dates_all = pd.concat([pd.read_parquet(f'files/ten_city/df_dates_{col_r}').rename({col_r: 'date_r'}, axis=1)[['rubricName', 'date_r', 'cnt', 'city', 'year_r']] for col_r in ['hour_r', 'date_r', 'week_r', 'week_day_r']])
df_dates_all['city_rubric'] = df_dates_all['city'] + '. ' + df_dates_all['rubricName']
df_dates_all = df_dates_all[~df_dates_all.rubricName.apply(lambda x: (x is None or x.isnumeric()))]
df_dates_all['cnt_all'] = df_dates_all.groupby('city_rubric')['cnt'].transform(np.sum)
df_dates_all['cnt_city_rubric'] = df_dates_all['cnt_all'].astype(str) + '_' + df_dates_all['city_rubric']
dfx = pd.pivot_table(df_dates_all, values='cnt', aggfunc='sum', index=['date_r'], columns=['cnt_city_rubric'])
t = dfx.corr().fillna(0).astype(np.float16)
#with open('t.pkl', 'wb') as fp:
#    pickle.dump(t, fp)

In [4]:
with open('t.pkl', 'rb') as fp:
    t = pickle.load(fp)
    
t = t.rename_axis([None]).rename_axis([None], axis=1)
t1 = t.stack()
t1 = t1.reset_index()
t1.columns = ['r1', 'r2', 'corr']
t1 = t1[t1.r1 != t1.r2]
t1['r1_cnt'] = t1['r1'].apply(lambda s: str(s).split('_')[0]).astype(int)
t1['r2_cnt'] = t1['r2'].apply(lambda s: str(s).split('_')[0]).astype(int)

#with open('t1.pkl', 'wb') as fp:
#    pickle.dump(t1, fp)

In [5]:
with open('t1.pkl', 'rb') as fp:
    t1 = pickle.load(fp)

#t1['cnt_diff'] = abs(t1.r1_cnt - t1.r2_cnt)
#t2 = t1[(t1.r1_cnt > 10000) & (t1.r2_cnt > 10000)]
#df_dates_all = pd.concat([pd.read_parquet(f'files/ten_city/df_dates_{col_r}').rename({col_r: 'date_r'}, axis=1)[['rubricName', 'date_r', 'cnt', 'city', 'year_r']] for col_r in ['hour_r', 'date_r', 'week_r', 'week_day_r']])
#df_dates_all['city_rubric'] = df_dates_all['city'] + '. ' + df_dates_all['rubricName']

In [ ]:
t1_m = t1[t1['corr'] < 0].sort_values(['cnt_diff', 'r1_cnt'], ascending=[True, False])
t1_z = t1[t1['corr'] == 0].sort_values(['cnt_diff', 'r1_cnt'], ascending=[True, False])

zero_corr = t1[t1['corr'] < abs(0.41)]
zero_corr_rubrics = pd.concat([zero_corr.r1, zero_corr.r2]).drop_duplicates().values
zero_corr_rubrics = [i.split('_')[1] for i in zero_corr_rubrics]

In [195]:
st1 = t2[t2['corr'] > 0.95].sample(1000)
st2 = t1[t1['corr'] < 0.95].sample(1000)
st3 = t1[(abs(t1['corr']) == 0)].sample(1000)
st4 = t1[(t1.r1_cnt > 15000000) & (t1.r2_cnt > 15000000)]

In [196]:
st1['group'] = 'corr>0.95'
st2['group'] = 'corr<0.95'
st3['group'] = 'corr=0'
st4['group'] = '>15 млн.запросов'

In [203]:
st = pd.concat([st1, st2, st3, st4])

In [204]:
st['r1'] = st['r1'].apply(lambda x: x.split('_')[1])
st['r2'] = st['r2'].apply(lambda x: x.split('_')[1])

In [206]:
st.to_excel('Таблица_Диплом_корреляции_230510_2345.xlsx')

In [215]:
t_al = t1[t1.r1.apply(lambda x: ('Астана.' in x)&('Косметика' in x))]

In [ ]:
t_al = t_al[t_al.r2.apply(lambda x: ('Астана.' not in x)&('Косметика' not in x))]

In [2]:
with open('t1.pkl', 'rb') as fp:
    t1 = pickle.load(fp)

In [26]:
t_al = t1[t1.r1.apply(lambda x: ('Ташкент.' in x)&('Мясо птицы' in x))]
t_al = t_al[t_al.r2.apply(lambda x: ('Ташкент.' not in x)&('Мясо птицы' not in x.lower()))]

In [27]:
t_al.shape

(12350, 5)

In [28]:
t_al.to_excel('t_al.xlsx', index=False)

In [91]:
fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")

negative_corr = t1_m.iloc[:1000]
negative_corr_rubrics = pd.concat([negative_corr.r1, negative_corr.r2]).drop_duplicates().values
negative_corr_rubrics = [i.split('_')[1] for i in negative_corr_rubrics]

df_dates_negative = df_dates_all[df_dates_all['city_rubric'].apply(lambda x: (x in negative_corr_rubrics[:25] ))]

dfx = pd.pivot_table(df_dates_negative, values='cnt', aggfunc='sum', index=['date_r'], columns=['city_rubric'])
t = dfx.corr().fillna(0)

sns.set_style({'font':fpath})
sns.heatmap(t, annot=False, cbar=False)

plt.show()

positive_corr = t2[t2['corr'] == 1]
positive_corr_rubrics = pd.concat([positive_corr.r1, positive_corr.r2]).drop_duplicates().values
positive_corr_rubrics = [i.split('_')[1] for i in positive_corr_rubrics]

df_dates_positive = df_dates_all[df_dates_all['city_rubric'].apply(lambda x: (x in positive_corr_rubrics[:12] ))]

dfx = pd.pivot_table(df_dates_positive, values='cnt', aggfunc='sum', index=['date_r'], columns=['city_rubric'])
t = dfx.corr().fillna(0)

#sns.heatmap(t, annot=True, cbar=False, annot_kws={'font':fpath})
sns.heatmap(t, annot=True, cbar=False, annot_kws={'font':fpath})
plt.show()

negative_corr = t1_m.iloc[:25]
negative_corr_rubrics = pd.concat([negative_corr.r1, negative_corr.r2]).drop_duplicates().values
negative_corr_rubrics = [i.split('_')[1] for i in negative_corr_rubrics]

df_dates_negative = df_dates_all[df_dates_all['city_rubric'].apply(lambda x: (x in negative_corr_rubrics[:12] ))]

dfx = pd.pivot_table(df_dates_negative, values='cnt', aggfunc='sum', index=['date_r'], columns=['city_rubric'])
t = dfx.corr().fillna(0)

sns.heatmap(t, annot=True, cbar=False)
plt.show()

df_dates_zero = df_dates_all[df_dates_all['city_rubric'].apply(lambda x: (x in zero_corr_rubrics[:12]))]

dfx = pd.pivot_table(df_dates_zero, values='cnt', aggfunc='sum', index=['date_r'], columns=['city_rubric'])
t = dfx.corr().fillna(0)

sns.heatmap(t, annot=True, cbar=False)
plt.show()